In [2]:
#Imports

import copy
import pandas as pd

# Requests sends and recieves HTTP requests.
import requests

# Beautiful Soup parses HTML documents in python.
from bs4 import BeautifulSoup
import csv
import pprint
import re

# Test URL Request Before Web Scraping  

In [5]:
#Request  Single URL for the Defensive Stats in year 2000
pfr_url = "https://www.pro-football-reference.com/years/2020/"
r = requests.get(pfr_url)

In [6]:
#If this returns 200 it was a success.
r.status_code

200

# Use API to Scrape Data

In [1]:
#Ben Kite Pro Football Reference WebScrape API
#https://github.com/BenKite/football_data/blob/master/profootballReferenceScrape.py
#This returns a list of table names needed to scrape the information from the desired table.
def findTables(url):
    res = requests.get(url)
    comm = re.compile("<!--|-->")
    soup = BeautifulSoup(comm.sub("", res.text), 'lxml')
    divs = soup.findAll('div', id = "content")
    divs = divs[0].findAll("div", id=re.compile("^all"))
    ids = []
    for div in divs:
        searchme = str(div.findAll("table"))
        x = searchme[searchme.find("id=") + 3: searchme.find(">")]
        x = x.replace("\"", "")
        if len(x) > 0:
            ids.append(x)
    return(ids)
print(findTables("https://www.pro-football-reference.com/years/2020/"))

NameError: name 'requests' is not defined

In [187]:
#This portion of Ben Kite's API will pull the information from the table in the list of table_ids
#The function is defined here and ran through a for loop with the desired years of the data saving each request to csv
#I made some modifications to add a year column and change the csv name based on the year of the data in the for loop
def pullTablestats(url, tableID, header = False):
    res = requests.get(url)
    ## Work around comments
    comm = re.compile("<!--|-->")
    soup = BeautifulSoup(comm.sub("", res.text), 'lxml')
    tables = soup.findAll('table', id = tableID)
    data_rows = tables[0].findAll('tr')
    game_data = [[td.getText() for td in data_rows[i].findAll(['th','td'])]
        for i in range(len(data_rows))
        ]
    data = pd.DataFrame(game_data)
    if header == True:
        data_header = tables[0].findAll('thead')
        data_header = data_header[0].findAll("tr")
        data_header = data_header[0].findAll("th")
        header = []
        for i in range(len(data.columns)):
            header.append(data_header[i].getText())
        data.columns = header
        data = data.loc[data[header[0]] != header[0]]
    data["year"]=year
    data = data.reset_index(drop = True)
    #writes to csv at the end of the function and uses fstring to change the year in the filename
    data.to_csv(f"team_offense{year}.csv")
    return(data)
   


In [189]:
#This will run the function to grab data for the range of years listed and save the results to csv.
for year in range(1935,2021):
    url="https://www.pro-football-reference.com/years/{}/opp.htm".format(year)
    pullTablestats(url,'team_stats',header=False)